In [28]:
pip install torch


  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/d6/a8/43e5033f9b2f727c158456e0720f870030ad3685c46f41ca3ca901b54922/torch-2.1.1-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/192.3 MB ? eta -:--:--
   ---------------------------------------- 0.1/192.3 MB ? eta -:--:--
   ---------------------------------------- 0.1/192.3 MB ? eta -:--:--
   ---------------------------------------- 0.1/192.3 MB ? eta -:--:--
   ---------------------------------------- 0.1/192.3 MB ? eta -:--:--
   ---------------------------------------- 0.1/192.3 MB ? eta -:--:--
   ---------------------------------------- 0.1/192.3 MB ? eta -:--:--
   ---------------------------------------- 0.1/192.3 MB 249.8 kB/s eta 0:12:50
   ---------------------------------------- 0.3/192.3 MB 873.8 kB/s eta 0:03:40
   ---------------------------------------- 0.5/192.3 MB 1.1 MB/s eta 0:02:49
   ---------------------------------------- 0.6/192.3 MB 1.3

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, mean_squared_error

In [22]:
df = pd.read_csv(r"C:\Users\ejanv\Downloads\RTD Self-Scheduled Nominations\consolidated_csv_selfsched_08SLWIND_G01.csv")

In [23]:
# transform date column into a datetime column
df['RUN_TIME'] = pd.to_datetime(df['RUN_TIME'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')
df = df.set_index('RUN_TIME')
df = df[~df.index.duplicated(keep='first')]
df.head()

,MKT_TYPE,REGION_NAME,ZONE_NAME,RESOURCE_NAME,TIME_INTERVAL,START_TIME,END_TIME,MP_NAME,SELF_SCHED_MW,Unnamed: 10
RUN_TIME,,,,,,,,,,
NaT,RTD,CVIS,PANAY,08SLWIND_G01,06/26/2021 12:05:00 AM,06/26/2021,06/26/2021 12:05:00 AM,TAREC,0.3,NaN
2021-06-26 00:05:00,RTD,CVIS,PANAY,08SLWIND_G01,06/26/2021 12:10:00 AM,06/26/2021 12:05:00 AM,06/26/2021 12:10:00 AM,TAREC,0.3,NaN
2021-06-26 00:10:00,RTD,CVIS,PANAY,08SLWIND_G01,06/26/2021 12:15:00 AM,06/26/2021 12:10:00 AM,06/26/2021 12:15:00 AM,TAREC,0.3,NaN
2021-06-26 00:15:00,RTD,CVIS,PANAY,08SLWIND_G01,06/26/2021 12:20:00 AM,06/26/2021 12:15:00 AM,06/26/2021 12:20:00 AM,TAREC,0.3,NaN
2021-06-26 00:20:00,RTD,CVIS,PANAY,08SLWIND_G01,06/26/2021 12:25:00 AM,06/26/2021 12:20:00 AM,06/26/2021 12:25:00 AM,TAREC,0.3,NaN


In [25]:
data = df[['SELF_SCHED_MW']].values.astype(float)
data = (data - np.min(data)) / (np.max(data) - np.min(data))
data

array([[0.00555556],
       [0.00555556],
       [0.00555556],
       ...,
       [0.0462963 ],
       [0.05185185],
       [0.05      ]])

In [26]:
# Split the time series data into training and testing sets
train_data = data[:int(len(data)*0.8)]
test_data = data[int(len(data)*0.8):]

In [29]:
import torch

# Convert the time series data to PyTorch tensors
train_data = torch.FloatTensor(train_data).unsqueeze(1)
test_data = torch.FloatTensor(test_data).unsqueeze(1)

In [30]:
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, input):
        batch_size = input.size(0)
        hidden = self.init_hidden(batch_size)
        lstm_out, hidden = self.lstm(input.view(len(input), batch_size, -1), hidden)
        output = self.fc(lstm_out[-1])
        return output

    def init_hidden(self, batch_size):
        return (torch.zeros(1, batch_size, self.hidden_size),
                torch.zeros(1, batch_size, self.hidden_size))

In [32]:
import torch.optim as optim

# Define the hyperparameters
input_size = ...
hidden_size = ...
output_size = ...
learning_rate = ...
num_epochs = ...

# Instantiate the model
model = LSTM(input_size, hidden_size, output_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))

TypeError: hidden_size should be of type int, got: ellipsis

In [ ]:
# Set the model to evaluation mode
model.eval()

# Define the test loop
test_loss = 0.0
with torch.no_grad():
    for inputs, labels in test_loader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

    # Compute the evaluation metrics
    avg_test_loss = test_loss / len(test_loader)
    print('Average Test Loss: {:.4f}'.format(avg_test_loss))

In [ ]:
# data1 = data.copy()
# data1['Date'] = data.index.date
# data1

In [ ]:
# max_quantity_per_day = data1.groupby('Date')['SELF_SCHED_MW'].max()
# data_max = pd.DataFrame(max_quantity_per_day).reset_index()
# data_max

In [ ]:
# data_max = data_max.set_index('Date')
# data_max.head()

In [ ]:
# data['SELF_SCHED_MW'] = data['SELF_SCHED_MW'].fillna(method='ffill')
# data[['SELF_SCHED_MW']].plot()

In [ ]:
# data_max['SELF_SCHED_MW'] = data_max['SELF_SCHED_MW'].fillna(method='ffill')
# data_max[['SELF_SCHED_MW']].plot()

In [ ]:
# from pmdarima.arima import ADFTest
# adf_test = ADFTest(alpha = 0.05)
# adf_test.should_diff(data)

In [ ]:
# adf_test = ADFTest(alpha = 0.05)
# adf_test.should_diff(data_max)

In [ ]:
# # Plot the time series while preserving the stepwise format
# data.plot(figsize=(10, 6))
# plt.title('Time Series with Stepwise Datetime Index')
# plt.xlabel('Datetime')
# plt.ylabel('Data')
# plt.grid(True)
# plt.show()

## LSTM trial